<a href="https://colab.research.google.com/github/yukuproj/made2/blob/main/Lab1_NLP_part1_Embedding_based_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages.

In [5]:
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
!gzip -d cc.uk.300.vec.gz

--2022-04-13 12:18:59--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  15.8MB/s    in 52s     

2022-04-13 12:19:52 (23.9 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]

--2022-04-13 12:20:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1257595219 (1.2G) [binary/octet-stream]
Saving to: ‘cc.uk.300

After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

In [ ]:
from gensim.models import KeyedVectors


uk_emb = KeyedVectors.load_word2vec_format("/content/cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("/content/cc.ru.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

In [7]:
august_embedding = ru_emb["август"]
august_embedding.shape, august_embedding[:5]

((300,), array([ 0.0033, -0.0322, -0.0519, -0.0808, -0.0131], dtype=float32))

In [8]:
ru_emb.most_similar([august_embedding])

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

The latter function also allows you to vary the amount of closest words via the `topn` argument:

In [9]:
ru_emb.most_similar([august_embedding], topn=3)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814)]

Another feature of `KeyedVectors` is that it allows to compute embeddings for multiple words simultaneously:

In [10]:
ru_emb[["август", "сентябрь"]].shape

(2, 300)

Everything above is true for the embeddings for Ukrainian language.

In [11]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

However, russian and ukrainian embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in Russian and Ukrainian:

In [12]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

## Translation

We'll build a simple translator, which will try to predict the russian embedding from the ukrainian one. For this we'll need a dataset of word pairs.

In [16]:
import pandas as pd




train_data_url = "uk_ru.train.tsv"
train_data = pd.read_csv(train_data_url, sep="\t", header=None)
train_data.columns = ["uk", "ru"]
print(f"Train dataset size: {len(train_data)}")

test_data_url = "uk_ru.test.tsv"
test_data = pd.read_csv(test_data_url, sep="\t", header=None)
test_data.columns = ["uk", "ru"]
print(f"Test dataset size: {len(test_data)}")

train_data.head()

Train dataset size: 1927
Test dataset size: 400


,uk,ru
0,iмовірно,вероятно
1,iснує,существует
2,iспит,экзамен
3,абияк,как-нибудь
4,або,или


Our method won't work with unknown words, so let's filter them out:

In [17]:
rows = []
for _, row in train_data.iterrows():
    if row["uk"] not in uk_emb or row["ru"] not in ru_emb:
        continue

    rows.append(row)

train_data = pd.DataFrame(rows)
print(f"Train dataset size: {len(train_data)}")

rows = []
for _, row in test_data.iterrows():
    if row["uk"] not in uk_emb or row["ru"] not in ru_emb:
        continue

    rows.append(row)

test_data = pd.DataFrame(rows)
print(f"Test dataset size: {len(test_data)}")

Train dataset size: 1880
Test dataset size: 393


We will train our model to predict embedding for the russian word from embedding of its ukrainian counterpart. For this reason we split our train and test data into ukrainian and russian words and compute corresponding embeddings to obtain `X` (ukrainian embeddings) and `y` (russian embeddings).

In [18]:
X_train, Y_train = uk_emb[train_data["uk"].values], ru_emb[train_data["ru"].values]
X_test, Y_test = uk_emb[test_data["uk"].values], ru_emb[test_data["ru"].values]

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language,

 and $y_i \in \mathrm{R}^d$ is the vector representation of its translation.
 
  Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. 
  
  Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

In [19]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
# mapping = ...

mapping = LinearRegression(fit_intercept= False).fit(X_train, Y_train)
W = mapping.coef_.T

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [20]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531432747840881),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884165763855),
 ('сентябрь', 0.8331484794616699),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278019428253174),
 ('ноябрь', 0.8243728280067444),
 ('июль', 0.8229618072509766),
 ('август', 0.8112280368804932),
 ('январь', 0.8022986650466919)]

In [ ]:
august

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in russian embedding space).

In [21]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    total = len(pairs)
    correct = 0
    for i in range(total):
        pair = pairs[i]
        predicted_vector = mapped_vectors[i]

        # YOUR CODE HERE
        if pair[1] in [x[0] for x in ru_emb.most_similar(predicted_vector.reshape(1, -1), topn=topn)]:
          correct = correct +  1

    return correct / total

In [22]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

Note that our `precision` function accepts lists of pairs of words, whereas we have dataframes. However, it is not a problem: we can get a list (actually, numpy array) of pairs via the `values` property.

In [23]:
assert precision(test_data.values, X_test) == 0.0
assert precision(test_data.values, Y_test) == 1.0

Let's see how well our model is doing.

In [24]:
top1 = precision(test_data.values, mapping.predict(X_test), 1)
print(f"Top-1 precision {100 * top1:.1f}%")

top5 = precision(test_data.values, mapping.predict(X_test), 5)
print(f"Top-5 precision {100 * top5:.1f}%")

Top-1 precision 62.8%
Top-5 precision 79.1%


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [26]:
import numpy as np
from numpy.linalg import svd

# YOUR CODE HERE
# Compute the orthogonal mapping (W^T)^* as defined in formula above.
# mapping = ...

u, s, vh = svd(np.matmul(X_train.T, Y_train), full_matrices=True)
mapping = np.matmul(u, vh)

Now our `mapping` is just a numpy array, meaning that it has no `predict` method. However, from the formulae above we know, that prediction is done using the matrix multiplication:

In [27]:
august = uk_emb["серпень"] @ mapping
ru_emb.most_similar([august])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938189506530762),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

Now let's compute our precision values and see, whether our trick did improve the results.

In [28]:
top1 = precision(test_data.values, X_test @ mapping, 1)
print(f"Top-1 precision {100 * top1:.1f}%")

top5 = precision(test_data.values, X_test @ mapping, 5)
print(f"Top-5 precision {100 * top5:.1f}%")

Top-1 precision 64.4%
Top-5 precision 79.9%


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [29]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
!gzip -d ./uk.txt.gz

--2022-04-13 12:52:45--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  2.27MB/s    in 0.8s    

2022-04-13 12:52:47 (2.27 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [30]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]
uk_corpus[:5]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n']

Now let's translate these sentences word-by-word. Before that, however, don't forget to tokenize your sentences. For that you may (or may not) find the `nltk.tokenize.WordPunctTokenizer` to be very useful.

In [31]:
from nltk.tokenize import WordPunctTokenizer

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []

    # YOUR CODE HERE
    tokenizer = WordPunctTokenizer()
    data_tok = [word.lower() for word in tokenizer.tokenize(sentence)]
    translated = ([ru_emb.most_similar([uk_emb[word] @ mapping], topn=1)[0][0] if word in uk_emb.vocab.keys() else '' for word in data_tok])
 

    return " ".join(translated)

In [32]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [35]:
for sent in uk_corpus[::10]:
    print(sent)
    print(translate(sent))
    print('\n')

Я вже закінчу коледж, коли ви вернетеся з Америки.

мной уже закончу колледж , когда мы прибежишь со америки .


Місто бомбардували ворожі літаки.

город бомбили враждебные самолеты .


Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.

возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .


Цього ранку випала роса.

этого утра выпала роса .


Біда не приходить одна.

беда не приходит одна .


Подивися на той дим.

посмотри по тот дым .


Я замовив два гамбургера.

мной заказал два гамбургера .


Я не хотів нікого образити.

мной не хотел никого обидеть .


Гора вкрита снігом.

гора покрыта снегом .


На фотографії в дівчини корона не з золота, а з квітів.

по фотографии во девушки корона не со золота , а со цветов .


У мене є мрія.

во меня То мечта .


Я приїхав у Японію з Китаю.

мной приехал во  со китая .


На півночі знаходиться Шотландія; на півдні — Англія; на заході — Уельс; і ще далі на заході — Північна Ірландія.


KeyboardInterrupt: ignored

Great! 
See second notebook for the Neural Machine Translation assignment.